### Note book Two

Compare the survey results for specific items indentified during shoreline surveillance of Lake Geneva. Group survey results in time by Winter, Spring, Summer or Fall, by body of water and municipality.

__Research question:__

_Does the data indicate that shoreline litter densities are seasonal? If so describe the relationship._

#### Contents

1. Automating some things
2. Making a MlwCode class  
3. Dealing with time
4. Grouping by season
5. Plot grouped by season and code


### Setting up

Make sure you are running the same packages as this notebook. There is a requirements.txt file for the virutal environment in the [repo](https://github.com/hammerdirt/SWE_2019.git). 

#### Imports

No changes form note book one

In [1]:
import numpy as np
import json
import csv
import datetime
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
import os
import matplotlib.dates as mdates
from matplotlib import colors as mcolors
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [2]:
# making the file structure
# if you changed the file structure then this function will be problematic


folders = ["Data", "Charts", "Utilities"]
here = os.getcwd()
# never comment this out -- it is used to save output
def make_folders():
    my_folders = {}
    for folder in folders:
        place = here +"/"+ folder
        my_folders[folder] = place
    return my_folders
my_folders = make_folders()

### Making the code class

Gather and compute details for the MLW Codes in a class. 

#### Utility functions

In [62]:
# nothign fancy, jsut storing the data we used in Notebook one in a class object.

# variables
variable_names=["beach_info", "get_toys", "get_sheeting"]
end_points = [
    "http://mwshovel.pythonanywhere.com/dirt/beaches/Lac-L%C3%A9man/",
    "http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/G32",
    "http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/G67",    
]

def get_the_data(variable_names,end_points):
    data = {}
    for i, name in enumerate(variable_names):
        data[name] = requests.get(end_points[i]).json()
    return data
# make a pieces per meter value for each observation
def get_pieces_per_meter(a_list_of_objects):
    new_list_of_objects = []
    the_keys = list(a_list_of_objects.keys())    
    for this_key in the_keys:
        new_list = a_list_of_objects[this_key]
        for the_object in new_list:
            new_object = the_object
            new_object["pcs_m"] = np.round(new_object["quantity"]/new_object["length"], 3)
            new_list_of_objects.append(new_object)
    return new_list_of_objects
# make a summary of the data 
def make_summary(data):
    results = {}
    pcs_m = []
    for result in data:
        pcs_m.append(result["pcs_m"])
    results["the_min"] = np.min(pcs_m)
    results["the_max"] = np.max(pcs_m)
    results["the_median"] = np.median(pcs_m)
    results["the_average"] = np.mean(pcs_m)
    results["twenty_fifth"] = np.percentile(pcs_m, 25)
    results["seventy_fifth"] =  np.percentile(pcs_m, 75)
    results["no_samples"] = len(pcs_m)
    results["last_sample"] = max([x["date"] for x in data])
    results["first_sample"] = min([x["date"] for x in data])
    return results
# make time value pairts
def time_value_pairs(aList):
    myPairs = [[result['date'], result['pcs_m']] for result in aList]
    return myPairs  
    

#### Wrting the class

In [75]:
code = "G32"
common_name = ["toys_data"]
end_point = ["http://mwshovel.pythonanywhere.com/dirt/codes/Lac-L%C3%A9man/" + code]

class MlWCodes():
    def __init__(self, code, common_names, end_points):
        self.code = code
        self.data = get_pieces_per_meter(get_the_data(common_names,end_points))
        self.time_value = time_value_pairs(self.data)
        self.summary = make_summary(self.data)
        
    
toys = MlWCodes(code=code, common_names=common_name, end_points=end_point)     

#### Using the class

In [79]:
# get the summary of data for toys
toys.summary

{'the_min': 0.012,
 'the_max': 2.222,
 'the_median': 0.062,
 'the_average': 0.12794666666666665,
 'twenty_fifth': 0.026500000000000003,
 'seventy_fifth': 0.125,
 'no_samples': 75,
 'last_sample': '2018-11-06',
 'first_sample': '2015-11-23'}